## Data Preprocessing

In [2]:
# Data = CMAPSSData
# Source = https://ti.arc.nasa.gov/tech/dash/groups/pcoe/prognostic-data-repository/
# Dataset = Turbofan Engine Degradation Simulation Data Set

In [3]:
# Data to be used - Train and Test data for 1

In [4]:
import pandas as pd
import numpy as np
import pickle as pkl
import matplotlib.pyplot as plt

### Notebook options

In [5]:
pd.set_option('display.max_colwidth', -1) # Column width
plt.rcParams['figure.figsize'] = [21, 10] # Size of the plots

### Data Directories

In [6]:
INPUT_DIR = '../data/raw/CMAPSSData/'
INT_DIR = '../data/interim/'
OUTPUT_DIR = '../data/interim/'

### Read the input data

In [7]:
input_file = OUTPUT_DIR + 'train_case2.csv'
df_data = pd.read_csv(input_file)

### Data Imputation

In [8]:
# Not required as there are no missing values
# Remove the additional columns

### Data Standardization

In [9]:
df_data.head()

,unit,time_cycles,setting1,setting2,setting3,meas01,meas02,meas03,meas04,meas05,...,meas12,meas13,meas14,meas15,meas16,meas17,meas18,meas19,meas20,meas21
0,1,1,34.9983,0.8400,100.0,449.44,555.32,1358.61,1137.23,5.48,...,183.06,2387.72,8048.56,9.3461,0.02,334,2223,100.00,14.73,8.8071
1,1,2,41.9982,0.8408,100.0,445.00,549.90,1353.22,1125.78,3.91,...,130.42,2387.66,8072.30,9.3774,0.02,330,2212,100.00,10.41,6.2665
2,1,3,24.9988,0.6218,60.0,462.54,537.31,1256.76,1047.45,7.05,...,164.22,2028.03,7864.87,10.8941,0.02,309,1915,84.93,14.08,8.6723
3,1,4,42.0077,0.8416,100.0,445.00,549.51,1354.03,1126.38,3.91,...,130.72,2387.61,8068.66,9.3528,0.02,329,2212,100.00,10.59,6.4701
4,1,5,25.0005,0.6203,60.0,462.54,537.07,1257.71,1047.93,7.05,...,164.31,2028.00,7861.23,10.8963,0.02,309,1915,84.93,14.13,8.5286


In [10]:
from sklearn.preprocessing import StandardScaler

exclude_cols = ['unit', 'time_cycles']

df_pca = df_data.copy()
features = [x for x in df_pca.columns.values if x not in set(exclude_cols)]
print(features)

# Separating out the features
x = df_pca.loc[:, features].values

# Standardizing the features
x = StandardScaler().fit_transform(x)

['setting1', 'setting2', 'setting3', 'meas01', 'meas02', 'meas03', 'meas04', 'meas05', 'meas06', 'meas07', 'meas08', 'meas09', 'meas10', 'meas11', 'meas12', 'meas13', 'meas14', 'meas15', 'meas16', 'meas17', 'meas18', 'meas19', 'meas20', 'meas21']


In [11]:
df_pca.head()

,unit,time_cycles,setting1,setting2,setting3,meas01,meas02,meas03,meas04,meas05,...,meas12,meas13,meas14,meas15,meas16,meas17,meas18,meas19,meas20,meas21
0,1,1,34.9983,0.8400,100.0,449.44,555.32,1358.61,1137.23,5.48,...,183.06,2387.72,8048.56,9.3461,0.02,334,2223,100.00,14.73,8.8071
1,1,2,41.9982,0.8408,100.0,445.00,549.90,1353.22,1125.78,3.91,...,130.42,2387.66,8072.30,9.3774,0.02,330,2212,100.00,10.41,6.2665
2,1,3,24.9988,0.6218,60.0,462.54,537.31,1256.76,1047.45,7.05,...,164.22,2028.03,7864.87,10.8941,0.02,309,1915,84.93,14.08,8.6723
3,1,4,42.0077,0.8416,100.0,445.00,549.51,1354.03,1126.38,3.91,...,130.72,2387.61,8068.66,9.3528,0.02,329,2212,100.00,10.59,6.4701
4,1,5,25.0005,0.6203,60.0,462.54,537.07,1257.71,1047.93,7.05,...,164.31,2028.00,7861.23,10.8963,0.02,309,1915,84.93,14.13,8.5286


### Writing standardized to file

In [12]:
std_file = OUTPUT_DIR + 'train_case1_std.pkl'
with open(std_file, 'wb') as f:
    pkl.dump(x, f, protocol=pkl.HIGHEST_PROTOCOL)

### Reading back from pickle

In [13]:
with open(std_file, 'rb') as f:
    x = pkl.load(f)

### PCA Projection

In [14]:
from sklearn.decomposition import PCA
num_components = len(df_pca.columns.values) - 2
pca = PCA(n_components=num_components)
principalComponents = pca.fit_transform(x)

In [15]:
sum = 0
idx = 1
print('| Component | Individual Contribution | Cumulative |')
print('| -- | -- | -- |')
for val in pca.explained_variance_ratio_:
    sum += val
    print('| Component ' + str(idx), end = ' | ')
    print("{:10.2f}".format(val*100), end=' | ')
    print("{:10.2f}".format(sum*100), end= ' |\n')
    idx += 1

| Component | Individual Contribution | Cumulative |
| -- | -- | -- |
| Component 1 |      77.35 |      77.35 |
| Component 2 |      19.96 |      97.31 |
| Component 3 |       1.91 |      99.22 |
| Component 4 |       0.41 |      99.62 |
| Component 5 |       0.17 |      99.79 |
| Component 6 |       0.10 |      99.89 |
| Component 7 |       0.07 |      99.96 |
| Component 8 |       0.03 |      99.98 |
| Component 9 |       0.01 |      99.99 |
| Component 10 |       0.00 |      99.99 |
| Component 11 |       0.00 |     100.00 |
| Component 12 |       0.00 |     100.00 |
| Component 13 |       0.00 |     100.00 |
| Component 14 |       0.00 |     100.00 |
| Component 15 |       0.00 |     100.00 |
| Component 16 |       0.00 |     100.00 |
| Component 17 |       0.00 |     100.00 |
| Component 18 |       0.00 |     100.00 |
| Component 19 |       0.00 |     100.00 |
| Component 20 |       0.00 |     100.00 |
| Component 21 |       0.00 |     100.00 |
| Component 22 |       0.00 |     10

### Create a principal dataframe with all the components

In [16]:
pca_col_list = ['pc'+str(i+1) for i in range(num_components)]
principalDf = pd.DataFrame(data = principalComponents, columns = pca_col_list)

### Find the influence of features in each component

In [17]:
df_infl = pd.DataFrame(pca.components_,columns=features,index=pca_col_list)

In [18]:
df_infl

,setting1,setting2,setting3,meas01,meas02,meas03,meas04,meas05,meas06,meas07,...,meas12,meas13,meas14,meas15,meas16,meas17,meas18,meas19,meas20,meas21
pc1,-1.918573e-01,-1.898804e-01,0.134141,2.051653e-01,2.273461e-01,2.304723e-01,2.313113e-01,2.023973e-01,2.112972e-01,2.207928e-01,...,2.208956e-01,1.341708e-01,1.714853e-01,-2.018051e-01,1.978547e-01,2.305683e-01,2.008428e-01,0.134141,2.183516e-01,2.183471e-01
pc2,-2.528392e-01,-2.248669e-01,-0.372245,1.950288e-01,4.928854e-02,-2.603471e-02,1.654839e-02,2.201963e-01,1.852538e-01,1.372298e-01,...,1.366101e-01,-3.722195e-01,-2.853409e-01,2.143180e-01,1.076223e-01,-2.387213e-02,-2.245897e-01,-0.372245,1.511926e-01,1.512212e-01
pc3,6.757462e-03,3.850051e-01,0.028096,2.553560e-01,2.351739e-01,1.253994e-01,-6.539875e-03,1.087197e-01,1.008249e-01,6.601313e-02,...,6.531707e-02,2.798432e-02,-2.874767e-01,1.776719e-01,-5.972289e-01,1.248774e-01,1.352747e-01,0.028096,7.389938e-02,7.391879e-02
pc4,-3.629464e-02,4.095069e-01,0.020667,1.469899e-01,1.359477e-01,9.544250e-02,8.244184e-03,-5.732009e-02,-6.659438e-02,-9.954069e-02,...,-1.002076e-01,2.094326e-02,-2.623034e-01,2.163077e-01,7.397053e-01,9.069014e-02,8.861036e-02,0.020667,-7.576903e-02,-7.570141e-02
pc5,2.465268e-01,-1.197215e-01,-0.204378,2.590742e-01,1.637613e-01,1.635845e-01,2.570327e-01,-1.204870e-01,-1.299240e-01,-1.413537e-01,...,-1.402874e-01,-2.012410e-01,4.699619e-01,2.883297e-01,-7.340406e-02,1.679546e-01,-6.083637e-02,-0.204378,-2.004068e-01,-2.000838e-01
pc6,-1.080103e-01,2.908622e-01,-0.018917,-9.529753e-02,-7.310972e-02,-1.045916e-01,-2.390185e-01,1.673215e-01,1.579189e-01,1.274365e-01,...,1.266681e-01,-1.138200e-02,6.694164e-01,9.624796e-02,1.092507e-01,-1.015218e-01,-5.799614e-02,-0.018917,1.596530e-01,1.598275e-01
pc7,-6.547685e-01,2.940172e-01,-0.041930,1.300132e-01,5.645368e-02,2.688902e-02,-5.731664e-02,-1.321884e-01,-1.620277e-01,-2.030116e-01,...,-2.055428e-01,-3.907142e-02,1.196287e-01,-4.173815e-04,-1.523377e-01,5.452228e-03,5.217237e-02,-0.041930,-6.771111e-02,-6.742924e-02
pc8,-3.055793e-01,1.671012e-01,0.057635,-4.412104e-01,-2.675007e-01,1.507929e-01,3.211007e-01,6.525064e-02,6.456262e-02,2.524900e-02,...,2.677276e-02,6.168516e-02,2.197511e-02,4.098294e-01,-6.922630e-02,1.366727e-01,-1.378303e-01,0.057635,4.538111e-02,4.640581e-02
pc9,-3.876454e-02,4.290555e-02,0.012261,5.359664e-02,5.215498e-02,-8.930721e-01,1.932243e-01,8.513223e-03,7.079717e-03,1.444599e-03,...,2.225848e-03,1.218125e-02,-6.824888e-04,3.432695e-02,5.984318e-05,3.749307e-01,3.531984e-02,0.012261,8.564763e-03,8.223005e-03
pc10,-1.977771e-02,8.348601e-02,0.027176,7.484208e-02,7.399735e-02,-1.561367e-01,6.291361e-01,1.428500e-02,1.338205e-02,7.852281e-03,...,7.619947e-03,2.731158e-02,1.758577e-03,6.380542e-03,2.084742e-03,-7.358427e-01,5.659850e-02,0.027176,1.332239e-02,1.841587e-02


### Write the principal components to file

In [19]:
principalDf_file = INT_DIR + 'fd002_components_' + str(num_components) + '.csv'
with open(principalDf_file, 'wb') as f:
    principalDf.to_csv(principalDf_file, header=True, index=False)

print('Writing to ', principalDf_file)

Writing to  ../data/interim/fd002_components_24.csv
